In [1]:
assay_index = 23
fp_index = 2

# Data download from GCP

In [2]:
import pandas as pd
!gsutil cp gs://chem_dsrc/P0_Toxcast/Data/assay_df/335_all_assay_df.csv ./

Copying gs://chem_dsrc/P0_Toxcast/Data/assay_df/335_all_assay_df.csv...
/ [1 files][  4.2 MiB/  4.2 MiB]                                                
Operation completed over 1 objects/4.2 MiB.                                      


In [3]:
# rdkit 2020.03.3 버전 다운로드
!pip install kora -q
import kora.install.rdkit
import numpy
import pandas
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from imblearn.combine import SMOTEENN 
from sklearn import metrics
import random
import os 
import time
import warnings
warnings.filterwarnings(action='ignore')


     |████████████████████████████████| 61kB 3.4MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
selected_assay_list = ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn','TOX21_AhR_LUC_Agonist', 'TOX21_Aromatase_Inhibition', 'TOX21_VDR_BLA_agonist_ratio', 'TOX21_VDR_BLA_antagonist_ratio', 'TOX21_AR_BLA_Agonist_ratio', 'TOX21_AR_BLA_Antagonist_ratio', 'TOX21_AR_LUC_MDAKB2_Agonist', 'TOX21_p53_BLA_p1_ratio', 'TOX21_p53_BLA_p2_ratio', 'TOX21_p53_BLA_p3_ratio', 'TOX21_p53_BLA_p4_ratio', 'TOX21_p53_BLA_p5_ratio', 'TOX21_HSE_BLA_agonist_ratio', 'TOX21_ELG1_LUC_Agonist', 'TOX21_ERa_BLA_Agonist_ratio', 'TOX21_ERa_BLA_Antagonist_ratio', 'TOX21_ESRE_BLA_ratio', 'TOX21_FXR_BLA_agonist_ratio', 'TOX21_FXR_BLA_antagonist_ratio', 'TOX21_GR_BLA_Agonist_ratio', 'TOX21_GR_BLA_Antagonist_ratio', 'TOX21_NFkB_BLA_agonist_ratio', 'TOX21_ARE_BLA_agonist_ratio', 'TOX21_TR_LUC_GH3_Agonist', 'TOX21_TR_LUC_GH3_Antagonist', 'TOX21_PPARg_BLA_Agonist_ratio', 'TOX21_PPARd_BLA_agonist_ratio', 'TOX21_PPARd_BLA_antagonist_ratio', 'TOX21_PPARg_BLA_antagonist_ratio']
selected_assay_list.remove('TOX21_NFkB_BLA_agonist_ratio')
selected_assay_list = [selected_assay_list[assay_index]]
ffpp_list = ['maccs', 'morgan','rdkit','pattern','layerd']
ffpp_list = [ffpp_list[fp_index]]

algo_list = ['MLP','GBT','RF','LR','KNN','NB']

dir_path = './result_sklearn_all_a_fp/'
os.mkdir(dir_path)
os.mkdir(dir_path + "metric/")

for i in ffpp_list : 
  os.mkdir(dir_path + "metric/"+i)

for i in ffpp_list : 
  for algo in algo_list :
    os.mkdir(dir_path + "metric/"+i+'/'+algo)



In [5]:
a_p_list = []
for p in ffpp_list :
  for a in selected_assay_list : 
    for algo in algo_list :
      a_p_list.append([p,a,algo]) 

assay_name = '335_all_assay'
df = pd.read_csv(assay_name+"_df.csv")

random.shuffle(a_p_list)
a_p_list

[['maccs', 'NVS_ADME_hCYP2B6', 'LR'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'NB'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'MLP'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'KNN'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'RF'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'GBT']]

In [6]:
def g_tf(ss_fp) : 

  
  # ss_fp = a_p_list[10]
  ffpp = ss_fp[0]#fingerfrint name
  ss =ss_fp[1]#assay name
  algo = ss_fp[2]

  start_time = time.time()
  print(ffpp ,algo, ss, 'start!_______________________________')

  df_ss = df.loc[:,['smiles',ss]]
  df_ss = df_ss.dropna()
  assays_ss = df_ss.columns

  #Smiles --> MACCS Keys
  assay = ss
  df_train  = df_ss[['smiles',ss] ]

  smiles_list = []
  err_smiles = []# fingerfrint로 변환시 에러 smiles
  fps = []
  enc_y = []
  for index, row in df_train.iterrows():

    if row[-1] == 'N' : 
      target = 0
    else :
      target = 1

    try : 
      mol = Chem.MolFromSmiles(row['smiles'])
      
      if ffpp == 'maccs' :    
          fp = MACCSkeys.GenMACCSKeys(mol)
      elif ffpp == 'morgan' : 
          fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2)
      elif ffpp == 'rdkit' : 
          fp = Chem.RDKFingerprint(mol)
      elif ffpp == 'pattern' : 
          fp = Chem.rdmolops.PatternFingerprint(mol)
      elif ffpp == 'layerd' : 
          fp = Chem.rdmolops.LayeredFingerprint(mol)
      else : 
          print("fingerfrint selection error!")
          
      smiles_list.append(row['smiles'])   
      fps.append(fp)
      enc_y.append(target)
    except : 
      err_smiles.append(row['smiles'])
      pass

  np_fps = []
  for fp in fps:
    arr = numpy.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    np_fps.append(arr)

  np_fps_array = numpy.array(np_fps)

  # print("변환 에러난 smiles 개수 : ", len(err_smiles), err_smiles)

  x_train_res, y_train_res = np_fps_array, np.array(enc_y)
  train = pd.DataFrame(data=x_train_res)
  train['label'] = y_train_res

  d = pd.DataFrame(columns = ['smiles','Type'] )
  d['smiles'] = smiles_list
  d['Type'] = enc_y

  #모든 물질에 대한 fingerprint를 string형태로 바꾼다(비교하기위함).
  mychems = []
  for item in np_fps:
      #print(item)
      item = np.array(item,dtype=int) #왜 굳이 두번? int형으로 추가하려면 이렇게 하면 되는구나....
      s = item.tolist() #넘파이어레이를 리스트로 바꾼다. 
      t = [str(i) for i in s]
      mychems.append(''.join(t))

  i = 0
  counter = 0
  chemdict = {} #딗셔너리.
  listofdups = []
  for item in mychems:
      try:
          chemdict[str(item)].append(i)
          #print('dup found')
          for j in range(len(chemdict[str(item)])):
              if (d['Type'][i] != d['Type'][chemdict[str(item)][j]]) :
                  listofdups.append(i)
                  counter+=1
      except: 
          #print('adding new')
          chemdict[str(item)] = []
          chemdict[str(item)].append(i)
      i+=1
  # print(str(counter)+' '+str(len(list(set(listofdups)))))
  #smote적용하기 전에서 listofdups를 빼면 된다!

  dups_list = list(set(listofdups))

  dups_list_0 = []
  for index in dups_list : 
    if train.iloc[index].label == 0 :
      dups_list_0.append(index)

  train = train.drop(dups_list_0)

  #분산이 0인 feature 제거
  train = train.loc[ :, train.std() > 0]



  # rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=36851234) 
  rskf = StratifiedKFold(n_splits = 10, random_state = 42)
  sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

  X = np.array( train.drop('label', axis =1) )
  y = np.array(train.label)

  count = 0 
  rep = 0

  metric_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])
  metric_fold_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])

  for train_index, test_index in rskf.split(X, y): 
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    if ss in ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn'] : 
      pass
    else : 
      print("SMOTEENN fitiing....",ffpp,algo,ss)
      X_train, y_train = sme.fit_sample(X_train, list(y_train) )
      print("....End fitiing",ffpp, ss)

    if algo == 'MLP' : 
      clf = MLPClassifier()
    if algo == 'GBT' : 
      clf = GradientBoostingClassifier()
    if algo == 'RF' : 
      clf = RandomForestClassifier()
    if algo == 'LR' : 
      clf = LogisticRegression()
    if algo == 'KNN' : 
      clf = KNeighborsClassifier()
    if algo == 'NB' : 
      clf = GaussianNB()

    clf.fit(X_train, y_train)

    result =  clf.predict_proba(X_test) # 첫 클래스의 예측확률, 두번째 클래스의 예측 확률 어떤것이 0, 1이지?
    y_pred = clf.predict(X_test)
    conf_mat = confusion_matrix(y_test,y_pred)
    tn, fp, fn, tp = conf_mat.ravel()

    auc =  roc_auc_score(y_test,result[:,1] )
    recall = tp/(tp+fn) # 실제 1인 것 중 모델이 1이라 한것의 비율
    specificity = tn / (fp+tn) # 실제 0인 것중 모델이 0이라한것의 비율
    b_a = (recall + specificity) / 2
    precision =  tp / (tp+fp)
    F1 = 2* (precision * recall) / (precision + recall)
    MCC = matthews_corrcoef(y_test,y_pred)
    accuracy = clf.score(X_test,y_test)


    metric_fold_df = metric_fold_df.append(pd.DataFrame([[auc,b_a,recall,precision,F1,MCC,accuracy]],columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy']) )
    count +=1
    # print(ffpp, ss, count)
    if count % 10 == 0 :
      average = metric_fold_df.mean()
      std = metric_fold_df.std()
      metric_fold_df = metric_fold_df.append(average.to_frame().T)
      metric_fold_df = metric_fold_df.append(std.to_frame().T)
      metric_fold_df = round(metric_fold_df ,4)
      metric_fold_df.index = [str(rep) +'_' +  str(i) for i in range(10)]+ [str(rep)+'_Mean' , str(rep)+'_SD']
      metric_df = metric_df.append(metric_fold_df)
      metric_fold_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])
      rep += 1
      # print(ffpp, ss, rep)
    

  #전체 평균내기.
  bool_list = [] 
  for i in metric_df.index :
    if'Mean' in i  or 'SD' in i : 
      bool_list.append(False) 
    else : 
      bool_list.append(True)

  average = metric_df[bool_list].mean().to_frame().T
  average.index = ['Mean']
  std = metric_df[bool_list].std().to_frame().T
  std.index =['SD']

  metric_df = metric_df.append(round(average,4) )
  metric_df = metric_df.append(round(std,4))
  metric_df.to_csv(dir_path+"metric/"+ffpp+'/'+algo+'/'+ss+"_metric.csv")
  # end_time = time.time()
  print(ffpp ,algo, ss, 'end','time spent',start_time - time.time())
  

In [7]:
import multiprocessing
CPU_CORE = multiprocessing.cpu_count() # 멀티프로세싱 CPU 사용 수
if __name__=='__main__':
    pool = multiprocessing.Pool(processes=CPU_CORE)
    pool.map(g_tf, a_p_list)
    pool.close()
    pool.join()  

maccs NB NVS_ADME_hCYP2B6 start!_______________________________
maccs MLP NVS_ADME_hCYP2B6 start!_______________________________
maccs LR NVS_ADME_hCYP2B6 start!_______________________________
maccs KNN NVS_ADME_hCYP2B6 start!_______________________________
maccs NB NVS_ADME_hCYP2B6 end time spent -0.7131412029266357
maccs RF NVS_ADME_hCYP2B6 start!_______________________________
maccs KNN NVS_ADME_hCYP2B6 end time spent -0.805941104888916
maccs GBT NVS_ADME_hCYP2B6 start!_______________________________
maccs LR NVS_ADME_hCYP2B6 end time spent -1.9110393524169922
maccs GBT NVS_ADME_hCYP2B6 end time spent -3.3654160499572754
maccs RF NVS_ADME_hCYP2B6 end time spent -4.5386035442352295
maccs MLP NVS_ADME_hCYP2B6 end time spent -6.365118026733398


In [10]:
a_p_list

[['maccs', 'NVS_ADME_hCYP2B6', 'LR'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'NB'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'MLP'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'KNN'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'RF'],
 ['maccs', 'NVS_ADME_hCYP2B6', 'GBT']]

In [11]:
for l in a_p_list : 
  csv_path = './result_sklearn_all_a_fp/metric/' + l[0] + '/' + l[2] + '/' + l[1] +'_metric.csv'
  gcp_path = 'gs://chem_dsrc/P0_Toxcast/result_sklearn_all_a_fp/metric/' + l[0] + '/' + l[2] + '/' + l[1] +'_metric.csv'
  !gsutil cp $csv_path $gcp_path

Copying file://./result_sklearn_all_a_fp/metric/maccs/LR/NVS_ADME_hCYP2B6_metric.csv [Content-Type=text/csv]...
/ [1 files][  736.0 B/  736.0 B]                                                
Operation completed over 1 objects/736.0 B.                                      
Copying file://./result_sklearn_all_a_fp/metric/maccs/NB/NVS_ADME_hCYP2B6_metric.csv [Content-Type=text/csv]...
/ [1 files][  720.0 B/  720.0 B]                                                
Operation completed over 1 objects/720.0 B.                                      
Copying file://./result_sklearn_all_a_fp/metric/maccs/MLP/NVS_ADME_hCYP2B6_metric.csv [Content-Type=text/csv]...
/ [1 files][  752.0 B/  752.0 B]                                                
Operation completed over 1 objects/752.0 B.                                      
Copying file://./result_sklearn_all_a_fp/metric/maccs/KNN/NVS_ADME_hCYP2B6_metric.csv [Content-Type=text/csv]...
/ [1 files][  750.0 B/  750.0 B]                             